In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Data

In [2]:
data_file_path = '../CRSP Data/exam_period_v6.csv'

In [3]:
df = pd.read_csv(data_file_path)
df.head()

C:\Users\User\AppData\Local\Temp\ipykernel_3800\4047245904.py:1: DtypeWarning: Columns (9,22,24,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file_path)


,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,CFACPR,CFACSHR,OPENPRC,NUMTRD,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,76932,2005-01-03,NaN,12,1,4619,29250N10,ENB,ENBRIDGE INC,NaN,...,4.0,4.0,49.40,NaN,-0.005223,-0.010034,-0.010048,-0.009869,-0.009901,-0.008119
1,76932,2005-01-04,NaN,12,1,4619,29250N10,ENB,ENBRIDGE INC,NaN,...,4.0,4.0,49.20,NaN,-0.016963,-0.012805,-0.012884,-0.014262,-0.014280,-0.011671
2,76932,2005-01-05,NaN,12,1,4619,29250N10,ENB,ENBRIDGE INC,NaN,...,4.0,4.0,48.30,NaN,-0.026705,-0.006124,-0.006195,-0.010687,-0.010734,-0.003628
3,76932,2005-01-06,NaN,12,1,4619,29250N10,ENB,ENBRIDGE INC,NaN,...,4.0,4.0,47.25,NaN,-0.003588,0.003483,0.003320,0.002287,0.002244,0.003506
4,76932,2005-01-07,NaN,12,1,4619,29250N10,ENB,ENBRIDGE INC,NaN,...,4.0,4.0,47.60,NaN,0.020123,-0.002217,-0.002222,-0.003704,-0.003718,-0.001431


In [4]:
## Check the trade status: Expect Active
df['TRDSTAT'].value_counts()

TRDSTAT
A    118090
Name: count, dtype: int64

In [5]:
## Check number of unique PERMCO
len(df['PERMCO'].unique())

48

In [6]:
## Check number of unique CUSIP
len(df['CUSIP'].unique())

49

In [7]:
## Identify duplicated PERMCO
tmp = df[['PERMCO', 'CUSIP']].drop_duplicates().groupby('PERMCO').count().reset_index()
tmp[tmp['CUSIP'] == 2]

,PERMCO,CUSIP
35,47003,2


In [8]:
## There are Shares in class A and B in the data, therefore class A will be select due to its higher market cap
tmp = df[df['PERMCO'] == 47003][['CUSIP', 'TICKER', 'SHRCLS', 'PRC', 'SHROUT']].drop_duplicates()
tmp['market_cap'] = tmp['PRC'] * tmp['SHROUT']
tmp.groupby('SHRCLS')['market_cap'].mean()

SHRCLS
A    2.613823e+07
B    7.588694e+06
Name: market_cap, dtype: float64

In [9]:
## Filter out Class B of PERMCO=47003
df = df[(df['PERMCO']!=47003) | 
        ((df['PERMCO']==47003) & (df['SHRCLS']=='A'))]

In [10]:
## Filter out negative prices
df = df[df['PRC'] >= 0]

In [11]:
## Clean data before calculation
df = df.dropna(subset=['TICKER', 'RET'])
df = df[df['RET'] != 'C']
df['RET'] = df['RET'].astype(float)

In [12]:
df.columns

Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP',
       'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT',
       'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT',
       'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG',
       'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM',
       'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD',
       'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD',
       'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')

# Calculate Adjusted Price

In [13]:
df['adjusted_prc'] = df['PRC'] / df['CFACPR']

# Calculate Weekly Return

In [14]:
## calulate the following friday
df['date'] = pd.to_datetime(df['date'])
df['fri_of_week'] = df['date'].dt.to_period('W-FRI').apply(lambda r: r.end_time).dt.date

## Before doing this step, first manually ensure the first monday does in in the prev year
df = df[(df['fri_of_week'].astype(str)>='2005-01-05') 
            & (df['fri_of_week'].astype(str)<='2015-12-31')]

In [18]:
## Calculate weekly return
df3 = df.copy()
df3['log_RET_plus1'] = np.log(df3['RET'] + 1)
df4 = df3.groupby(['PERMCO','fri_of_week']).agg({'log_RET_plus1': 'sum', 
                                                                     'TICKER': lambda x: list(set(x))[0], 
                                                                     'COMNAM': lambda x: list(set(x))[0]})
df4 = df4.reset_index().rename(columns={'log_RET_plus1': 'sum_log_RET_plus1'})
df4['weekly_return'] = (np.exp(df4['sum_log_RET_plus1']) - 1) * 100
df4.head()

,PERMCO,fri_of_week,sum_log_RET_plus1,TICKER,COMNAM,weekly_return
0,2381,2005-01-07,-0.033085,ENB,ENBRIDGE INC,-3.254320
1,2381,2005-01-14,0.004351,ENB,ENBRIDGE INC,0.436058
2,2381,2005-01-21,0.013756,ENB,ENBRIDGE INC,1.385143
3,2381,2005-01-28,0.028941,ENB,ENBRIDGE INC,2.936365
4,2381,2005-02-04,0.015140,ENB,ENBRIDGE INC,1.525478


In [19]:
df4.shape

(23886, 6)

In [20]:
## Check how many weeks each stock has
df4.groupby('PERMCO')['fri_of_week'].count().reset_index().groupby('fri_of_week').count().reset_index()

,fri_of_week,PERMCO
0,165,1
1,167,1
2,191,1
3,195,1
4,234,1
5,254,1
6,264,1
7,316,1
8,322,1
9,380,1


In [21]:
## Get the list of stocks having >= 573 weeks
tmp = df4.groupby('PERMCO')['fri_of_week'].count().reset_index()
tmp = tmp[tmp['fri_of_week']>=573]
tmp.head()

,PERMCO,fri_of_week
0,2381,573
1,11937,573
2,15434,573
3,20064,573
4,20253,573


In [22]:
## Filter Stocks having >= 572 weeks
df4_non_missing = pd.merge(df4, tmp[['PERMCO']], on='PERMCO', how='inner')

In [23]:
## Check the oldest and newest friday of week in the data
print(df4_non_missing['fri_of_week'].min(), df4_non_missing['fri_of_week'].max())

2005-01-07 2015-12-25


In [24]:
## Drop duplicated (PERMCO, fri-of-week) in which keeping the first row
df4_non_missing = df4_non_missing.drop_duplicates(subset=['PERMCO', 'fri_of_week'])

In [26]:
## Add a column of the return sign
df4_non_missing['weekly_return_sign'] = np.sign(df4_non_missing['weekly_return'])
df4_non_missing['weekly_return_sign'] = df4_non_missing['weekly_return_sign'].apply(lambda x: 1 if x > 0 else 0)
df4_non_missing.head()

,PERMCO,fri_of_week,sum_log_RET_plus1,TICKER,COMNAM,weekly_return,weekly_return_sign
0,2381,2005-01-07,-0.033085,ENB,ENBRIDGE INC,-3.254320,0
1,2381,2005-01-14,0.004351,ENB,ENBRIDGE INC,0.436058,1
2,2381,2005-01-21,0.013756,ENB,ENBRIDGE INC,1.385143,1
3,2381,2005-01-28,0.028941,ENB,ENBRIDGE INC,2.936365,1
4,2381,2005-02-04,0.015140,ENB,ENBRIDGE INC,1.525478,1


In [27]:
df4_non_missing.groupby('PERMCO')['TICKER'].count().reset_index().groupby('TICKER').count()

,PERMCO
TICKER,
573,34


In [28]:
## Sort to ensure proper order
df4_non_missing.sort_values(by=['PERMCO', 'fri_of_week'], inplace=True)

## Calculate the moving return for each ticker
df4_non_missing['previous_weekly_return'] = df4_non_missing.groupby(['PERMCO'])['weekly_return'].shift(1)
df4_non_missing['moving_return'] = df4_non_missing['weekly_return'] - df4_non_missing['previous_weekly_return']

## Add a column of the moving return direction
df4_non_missing['moving_return_dir'] = np.sign(df4_non_missing['moving_return'])
df4_non_missing['moving_return_dir'] = df4_non_missing['moving_return_dir'].apply(lambda x: 1 if x > 0 else 0)

In [29]:
df4_non_missing.groupby('PERMCO')['TICKER'].count().reset_index().groupby('TICKER').count()

,PERMCO
TICKER,
573,34


In [41]:
## Calculate adjusted price at the end of the week
df5 = df.copy()

## Select eow row
df5 = df5.sort_values(by=['PERMCO', 'date'], ascending=True).groupby(['PERMCO', 'fri_of_week']).last().reset_index()

## Filter Stocks having >= 572 weeks
df5_non_missing = pd.merge(df5, tmp[['PERMCO']], on='PERMCO', how='inner')

df5_non_missing = df5_non_missing[['PERMCO', 'fri_of_week', 'adjusted_prc']]
df5_non_missing = df5_non_missing.rename(columns={'adjusted_prc': 'eow_adjusted_prc'})

In [42]:
df5_non_missing.groupby('PERMCO')['fri_of_week'].count().reset_index().groupby('fri_of_week').count()

,PERMCO
fri_of_week,
573,34


In [43]:
## Add a column of weekly eow adjusted price
df6 = pd.merge(df4_non_missing, df5_non_missing, on=['PERMCO', 'fri_of_week'], how='inner')

In [44]:
## Sort to ensure proper order
df6.sort_values(by=['PERMCO', 'fri_of_week'], inplace=True)

## Calculate the moving return for each ticker
df6['previous_eom_adjusted_prc'] = df6.groupby(['PERMCO'])['eow_adjusted_prc'].shift(1)
df6['moving_prc'] = df6['eow_adjusted_prc'] - df6['previous_eom_adjusted_prc']

## Add a column of the moving return direction
df6['moving_prc_dir'] = np.sign(df6['moving_prc'])
df6['moving_prc_dir'] = df6['moving_prc_dir'].apply(lambda x: 1 if x > 0 else 0)

In [45]:
df6.groupby('PERMCO')['TICKER'].count().reset_index().groupby('TICKER').count()

,PERMCO
TICKER,
573,34


In [46]:
## Remove data of missing return or moving prc
df7 = df6[~df6['moving_return'].isna() & ~df6['moving_prc'].isna()]

In [47]:
## Check number of unique PERMCO
df7['PERMCO'].nunique()

34

In [48]:
df7.head()

,PERMCO,fri_of_week,sum_log_RET_plus1,TICKER,COMNAM,weekly_return,weekly_return_sign,previous_weekly_return,moving_return,moving_return_dir,eow_adjusted_prc,previous_eom_adjusted_prc,moving_prc,moving_prc_dir
1,2381,2005-01-14,0.004351,ENB,ENBRIDGE INC,0.436058,1,-3.254320,3.690378,1,12.0925,12.0400,0.0525,1
2,2381,2005-01-21,0.013756,ENB,ENBRIDGE INC,1.385143,1,0.436058,0.949086,1,12.2600,12.0925,0.1675,1
3,2381,2005-01-28,0.028941,ENB,ENBRIDGE INC,2.936365,1,1.385143,1.551221,1,12.6200,12.2600,0.3600,1
4,2381,2005-02-04,0.015140,ENB,ENBRIDGE INC,1.525478,1,2.936365,-1.410887,0,12.8125,12.6200,0.1925,1
5,2381,2005-02-11,0.003782,ENB,ENBRIDGE INC,0.378910,1,1.525478,-1.146567,0,12.7600,12.8125,-0.0525,0


In [49]:
df7.groupby('PERMCO')['TICKER'].count().reset_index().groupby('TICKER').count()

,PERMCO
TICKER,
572,34


In [50]:
## Save preprocessed data to csbv
df7.to_csv('./exam_period_v6_preprocessed_nonMissing_14Jun24.csv', index=False)

In [51]:
## Check available columns for future use
df7.columns

Index(['PERMCO', 'fri_of_week', 'sum_log_RET_plus1', 'TICKER', 'COMNAM',
       'weekly_return', 'weekly_return_sign', 'previous_weekly_return',
       'moving_return', 'moving_return_dir', 'eow_adjusted_prc',
       'previous_eom_adjusted_prc', 'moving_prc', 'moving_prc_dir'],
      dtype='object')

In [52]:
## Check the number of weeks of each stock
df7.groupby('PERMCO')['fri_of_week'].count().reset_index().groupby('fri_of_week').count()

,PERMCO
fri_of_week,
572,34


In [53]:
## Check the info of each stock
df7[['TICKER', 'COMNAM', 'PERMCO']].drop_duplicates().sort_values(by=['PERMCO'])

,TICKER,COMNAM,PERMCO
1,ENB,ENBRIDGE INC,2381
574,CHK,CHESAPEAKE ENERGY CORP,11937
1147,LNG,CHENIERE ENERGY INC,15434
1720,AHC,AMERADA HESS CORP,20064
1789,HES,HESS CORP,20064
2293,BHI,BAKER HUGHES INC,20253
2866,BP,B P PLC,20333
3439,CVX,CHEVRONTEXACO CORP,20440
3456,CVX,CHEVRON CORP NEW,20440
4225,EQT,E Q T CORP,20662


## Select top 25 tickers by market cap

In [54]:
## Load the data and calculate daily market cap
mkt_cap_df = pd.read_csv(data_file_path)
mkt_cap_df['market_cap'] = (mkt_cap_df['PRC'].abs() / mkt_cap_df['CFACPR']) * (mkt_cap_df['SHROUT'] * mkt_cap_df['CFACSHR'] )
mkt_cap_df = mkt_cap_df[['PERMCO', 'market_cap']]

C:\Users\User\AppData\Local\Temp\ipykernel_3800\1033636491.py:2: DtypeWarning: Columns (9,22,24,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  mkt_cap_df = pd.read_csv(data_file_path)


In [55]:
## Calculate average market cap over years
mkt_cap_df = mkt_cap_df.groupby(['PERMCO'])['market_cap'].mean().reset_index()
mkt_cap_df.head()

,PERMCO,market_cap
0,2381,2.261126e+07
1,11937,1.378532e+07
2,15434,4.303639e+06
3,20064,2.019897e+07
4,20253,2.167095e+07


In [56]:
## Select stock in energy sectors
mkt_cap_df = pd.merge(mkt_cap_df.drop_duplicates(), df7[['PERMCO']].drop_duplicates(), how='inner', on='PERMCO')

In [57]:
## Select top 25 market cap
mkt_cap_25 = mkt_cap_df.sort_values(by=['market_cap'], ascending=False).head(25)
mkt_cap_25.shape

(25, 2)

In [58]:
## Merge data
df8 = pd.merge(df7, mkt_cap_25, how='inner', on='PERMCO')

In [59]:
## Check the number of weeks of each stock
df8.groupby('PERMCO')['fri_of_week'].count().reset_index().groupby('fri_of_week').count()

,PERMCO
fri_of_week,
572,25


In [60]:
## Observe the example of the data
df8.head()

,PERMCO,fri_of_week,sum_log_RET_plus1,TICKER,COMNAM,weekly_return,weekly_return_sign,previous_weekly_return,moving_return,moving_return_dir,eow_adjusted_prc,previous_eom_adjusted_prc,moving_prc,moving_prc_dir,market_cap
0,2381,2005-01-14,0.004351,ENB,ENBRIDGE INC,0.436058,1,-3.254320,3.690378,1,12.0925,12.0400,0.0525,1,2.261126e+07
1,2381,2005-01-21,0.013756,ENB,ENBRIDGE INC,1.385143,1,0.436058,0.949086,1,12.2600,12.0925,0.1675,1,2.261126e+07
2,2381,2005-01-28,0.028941,ENB,ENBRIDGE INC,2.936365,1,1.385143,1.551221,1,12.6200,12.2600,0.3600,1,2.261126e+07
3,2381,2005-02-04,0.015140,ENB,ENBRIDGE INC,1.525478,1,2.936365,-1.410887,0,12.8125,12.6200,0.1925,1,2.261126e+07
4,2381,2005-02-11,0.003782,ENB,ENBRIDGE INC,0.378910,1,1.525478,-1.146567,0,12.7600,12.8125,-0.0525,0,2.261126e+07


In [61]:
## Check the number of unique PERMCO
df8['PERMCO'].nunique()

25

In [62]:
## Save data to csv
df8.to_csv('exam_period_v6_preprocessed_top25_14Jun24.csv', index=False)

In [63]:
## Check stock info
df8[['TICKER', 'COMNAM', 'PERMCO']].drop_duplicates().sort_values(by='PERMCO')

,TICKER,COMNAM,PERMCO
0,ENB,ENBRIDGE INC,2381
572,CHK,CHESAPEAKE ENERGY CORP,11937
1144,AHC,AMERADA HESS CORP,20064
1213,HES,HESS CORP,20064
1716,BHI,BAKER HUGHES INC,20253
2288,BP,B P PLC,20333
2860,CVX,CHEVRONTEXACO CORP,20440
2877,CVX,CHEVRON CORP NEW,20440
3432,XOM,EXXON MOBIL CORP,20678
4004,HAL,HALLIBURTON CO,20868
